# Homework 3

## Basic

In [27]:
import numpy as np

In [28]:
def cholesky(A):
    n = len(A)

    # Create zero matrix for L
    L = [[0.0] * n for i in range(n)]

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                # LaTeX: l_{kk} = \sqrt{ a_{kk} - \sum^{k-1}_{j=1} l^2_{kj}}
                L[i][k] = np.sqrt(A[i][i] - tmp_sum)
            else:
                # LaTeX: l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum^{k-1}_{j=1} l_{ij} l_{kj} \right)
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

In [40]:
def value_hw3(K, r, T, num, repeat, n, S : list, q: list, sigma: list, rho: list):
    sigma = np.reshape(sigma, (len(sigma), 1))
    cov_mat = sigma * sigma.T * np.array(rho)
    A = np.array(cholesky(cov_mat)).T
    V_n = []
    for j in range(repeat):
        V_list = []
        for i in range(num):
            z_T = np.matmul(np.random.randn(1, n) ,A) * np.sqrt(T)
            S_list = np.exp(np.log(S) + (r - np.array(q) - 0.5 * np.array(sigma) ** 2) * T + z_T)
            V_list.append((np.amax(S_list) - K) * (1 if np.amax(S_list) > K else 0))
        V_n.append(np.exp(-r * T) * np.sum(V_list) / num)
    V_n = np.array(V_n)
    return V_n.mean(), V_n.std(), [V_n.mean() - 2 * V_n.std(), V_n.mean() + 2 * V_n.std()]

In [39]:
value_hw3(100, 0.1, 0.5, 10000, 20, 2, [95]*2, [0.05]*2, [0.5]*2, [[1, 1], [1, 1]])

(11.914928386489136,
 0.20369020877090435,
 [11.507547968947327, 12.322308804030945])

In [41]:
value_hw3(100, 0.1, 0.5, 10000, 20, 2, [95]*2, [0.05]*2, [0.5]*2, [[1, -1], [-1, 1]])

(23.885516660988863,
 0.20817067720955085,
 [23.46917530656976, 24.301858015407966],
 array([[ 0.5, -0.5],
        [ 0. ,  0. ]]))

In [42]:
value_hw3(100, 0.1, 0.5, 10000, 20, 5, [95]*5, [0.05]*5, [0.5]*5, np.ones([5, 5]) * 0.5 + np.eye(5) * 0.5)

(30.379798520299552,
 0.3364210982102388,
 [29.706956323879073, 31.05264071672003],
 array([[0.5       , 0.25      , 0.25      , 0.25      , 0.25      ],
        [0.        , 0.4330127 , 0.14433757, 0.14433757, 0.14433757],
        [0.        , 0.        , 0.40824829, 0.10206207, 0.10206207],
        [0.        , 0.        , 0.        , 0.39528471, 0.07905694],
        [0.        , 0.        , 0.        , 0.        , 0.38729833]]))

## Bonus 1

In [52]:
def value_hw3_bonus(K, r, T, num, repeat, n, S : list, q: list, sigma: list, rho: list):
    sigma = np.reshape(sigma, (len(sigma), 1))
    cov_mat = sigma * sigma.T * np.array(rho)
    A = np.array(cholesky(cov_mat)).T
    V_n = []
    for j in range(repeat):
        V_list = []
        rand_list = np.random.randn(int(num/2), n)
        rand_list = np.append(rand_list, -rand_list)
        rand_list = rand_list / rand_list.std()
        for i in range(num):
            z_T = np.matmul(rand_list[i * n : (i+1) * n], A) * np.sqrt(T)
            S_list = np.exp(np.log(S) + (r - np.array(q) - 0.5 * np.array(sigma) ** 2) * T + z_T)
            V_list.append((np.amax(S_list) - K) * (1 if np.amax(S_list) > K else 0))
        V_n.append(np.exp(-r * T) * np.sum(V_list) / num)
    V_n = np.array(V_n)
    return V_n.mean(), V_n.std(), [V_n.mean() - 2 * V_n.std(), V_n.mean() + 2 * V_n.std()]

In [53]:
value_hw3_bonus(100, 0.1, 0.5, 10000, 20, 5, [95]*5, [0.05]*5, [0.5]*5, np.ones([5, 5]) * 0.5 + np.eye(5) * 0.5)

(30.388845693200647,
 0.07841189179937748,
 [30.23202190960189, 30.545669476799404])

## Bonus 2

In [50]:
def value_hw3_bonus2(K, r, T, num, repeat, n, S : list, q: list, sigma: list, rho: list):
    sigma = np.reshape(sigma, (len(sigma), 1))
    cov_mat = sigma * sigma.T * np.array(rho)
    A = np.array(cholesky(cov_mat)).T
    V_n = []
    for j in range(repeat):
        V_list = []
        rand_list = np.random.randn(num, n)
        rand_ = rand_list - (np.sum(rand_list, axis=0) / num)
        rand_cov = np.cov(rand_.T)
        A_ = np.array(cholesky(rand_cov)).T
        A_inv = np.linalg.inv(A_)
        rand__ = np.matmul(A_inv, rand_.T)
        for i in range(num):
            z_T = np.matmul(rand__[:, i], A) * np.sqrt(T)
            S_list = np.exp(np.log(S) + (r - q[0] - 0.5 * (sigma[0] ** 2)) * T + z_T)
            V_list.append(np.amax((S_list - K) * (S_list > K)))
#             print(z_T, S_list)
        V_n.append(np.exp(-r * T) * np.sum(V_list) / num)
    V_n = np.array(V_n)
    return V_n.mean(), V_n.std(), [V_n.mean() - 2 * V_n.std(), V_n.mean() + 2 * V_n.std()], V_n

In [54]:
value_hw3_bonus2(100, 0.1, 0.5, 10000, 20, 5, [95]*5, [0.05]*5, [0.5]*5, np.ones([5, 5]) * 0.5 + np.eye(5) * 0.5)

(30.363123366215955,
 0.1430324697844082,
 [30.077058426647138, 30.649188305784772],
 array([30.33557913, 30.20028155, 30.50242202, 30.53813042, 30.37810417,
        30.35793285, 30.49069028, 30.43878968, 30.05578854, 30.27975432,
        30.71097694, 30.22967765, 30.41186784, 30.38661238, 30.17380096,
        30.45206789, 30.2942672 , 30.41236498, 30.23895429, 30.37440423]))